In [111]:
!pip install pyspark

In [112]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg

# Initialize a Spark session
spark = SparkSession.builder.appName("StoreSalesAnalysis").getOrCreate()


sales_data = spark.read.option("header", "true" )\
                                              .option("inferSchema", "true" )\
                                              .csv("/content/store_sales_record.csv")


item_price = spark.read.option("header", "true" )\
                                              .option("inferSchema", "true" )\
                                              .option('delimiter',"\t")\
                                              .csv("/content/item_prices.csv")

sales_df=sales_data.join(item_price, item_price.ItemID==sales_data.ItemID, 'inner').drop(sales_data.ItemID)
sales_df = sales_df.withColumn('revenue', sales_df.Price *  sales_df.sold)
sales_df.show()

+-------+----+------+-----+-------+
|StoreID|sold|ItemID|Price|revenue|
+-------+----+------+-----+-------+
|     S1|  10|     a|  150|   1500|
|     S1|  20|     b|  280|   5600|
|     S1|   5|     c|   90|    450|
|     S1|   8|     d|  420|   3360|
|     S1|  15|     e|  200|   3000|
|     S1|  12|     f|  160|   1920|
|     S1|  18|     g|  310|   5580|
|     S1|   7|     h|   80|    560|
|     S1|  25|     i|  380|   9500|
|     S1|  14|     j|  290|   4060|
|     S2|   5|     a|  150|    750|
|     S2|  22|     b|  280|   6160|
|     S2|   8|     c|   90|    720|
|     S2|  13|     d|  420|   5460|
|     S2|  16|     e|  200|   3200|
|     S2|  10|     f|  160|   1600|
|     S2|  25|     g|  310|   7750|
|     S2|   6|     h|   80|    480|
|     S2|  20|     i|  380|   7600|
|     S2|  18|     j|  290|   5220|
+-------+----+------+-----+-------+
only showing top 20 rows



In [113]:
#total sales of each store
total_sales_store =sales_df.groupBy("StoreID").agg(sum("revenue").alias("TotalSalesPerStore"))
total_sales_store.show()


+-------+------------------+
|StoreID|TotalSalesPerStore|
+-------+------------------+
|    S10|             38630|
|     S6|             38270|
|     S8|             38990|
|     S7|             39300|
|     S4|             33260|
|     S3|             38040|
|     S5|             39820|
|     S2|             38940|
|     S9|             31480|
|     S1|             35530|
+-------+------------------+



In [114]:
#total number sold of each item
total_sales_item =sales_data.groupBy("ItemID").agg(sum("sold").alias("TotalsoldPerItem"))
total_sales_item.show()

+------+----------------+
|ItemID|TotalsoldPerItem|
+------+----------------+
|     g|             206|
|     f|             125|
|     e|             139|
|     h|              99|
|     d|             156|
|     c|             111|
|     i|             170|
|     j|             153|
|     b|             175|
|     a|             128|
+------+----------------+



In [115]:
#the average total sales
total_item_store=sales_data.groupBy("StoreID").agg(sum("sold").alias("Total_item_PerStore"))
avg_total_sales=total_sales_store.join(total_item_store, total_sales_store.StoreID==total_item_store.StoreID, 'inner').drop(total_item_store.StoreID)
avg_total_sales=avg_total_sales.withColumn('Average', avg_total_sales.TotalSalesPerStore / avg_total_sales.Total_item_PerStore)
avg_total_sales.select("StoreID","Average").show()

+-------+------------------+
|StoreID|           Average|
+-------+------------------+
|    S10| 261.0135135135135|
|     S6|251.77631578947367|
|     S8|259.93333333333334|
|     S7|260.26490066225165|
|     S4|230.97222222222223|
|     S3| 264.1666666666667|
|     S5|265.46666666666664|
|     S2| 272.3076923076923|
|     S9| 215.6164383561644|
|     S1|265.14925373134326|
+-------+------------------+



In [116]:
#grand total sales of  all stores
grand_total_sales=total_sales_store.agg(sum("TotalSalesPerStore").alias("TotalSalesAllStores"))
grand_total_sales.show()

+-------------------+
|TotalSalesAllStores|
+-------------------+
|             372260|
+-------------------+

